In [ ]:
import pathlib
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

### Pre-Processing

In [ ]:
MAIN_FOLDER = pathlib.Path("New_Section")
TRAIN_FILE = MAIN_FOLDER / "data/CaseI-Attacks without any change.csv"
TEST_FILE = MAIN_FOLDER / "data/CaseIII-Attacks after DG integration.csv"

In [ ]:
processed_train_file = MAIN_FOLDER / "data/case1_train.csv"
processed_test_file = MAIN_FOLDER / "data/case3_test.csv"

def catter(label):
    if label == "Normal":
        return 0
    else:
        num = [int(x) for x in label.split() if x.isdigit()]
        return num[0]

def label_fix(label):
    if label == 1010:
        return 10
    elif label == 1111:
        return 11
    elif label == 1212:
        return 12
    elif label == 1313:
        return 13
    elif label == 1414:
        return 14
    else:
        return label
    

train_pd = pd.read_csv(TRAIN_FILE)
print(train_pd[train_pd.isna().any(axis=1)])
train_pd["Label"] = train_pd["Label"].apply(catter)
train_pd.describe()

In [ ]:
test_pd = pd.read_csv(TEST_FILE)
print(test_pd[test_pd.isna().any(axis=1)])
test_pd["Label"] = test_pd["Label"].apply(catter)
test_pd["Label"] = test_pd["Label"].apply(label_fix)
test_pd.describe()

In [ ]:
train_pd.to_csv(processed_train_file)
test_pd.to_csv(processed_test_file)

In [ ]:
print(train_pd.Label.unique())
print(test_pd.Label.unique())

In [ ]:
def united_cat(df):
    unq_labels = sorted(df["Label"].unique())
    label_dict = dict(zip(unq_labels, list(range(len(unq_labels)))))
    df.Label = df["Label"].map(label_dict)
    return df, label_dict

new_train_df, train_label_dict = united_cat(train_pd)
new_test_df, test_label_dict = united_cat(test_pd)

In [ ]:
y_train = new_train_df.pop("Label")
y_train = to_categorical(y_train)
x_train = new_train_df

y_test= new_test_df.pop("Label")
y_test = to_categorical(y_test)
x_test = new_test

In [ ]:
mean_x = np.mean(x_train, axis=0)
var_x = np.var(x_train, axis=0)
print(mean_x, var_x)

In [ ]:
x_stats = pd.read_csv(MAIN_FOLDER / "data/x_stats.csv", index_col=0)
x_norm_layer = tf.keras.layers.experimental.preprocessing.Normalization(mean=x_stats.loc['mean'], variance=x_stats.loc['var'])
x_train_norm = x_norm_layer(x_train)
x_test_norm = x_norm_layer(x_test)